In [1]:
import numpy as np
import pandas as pd
import os
import datetime
import csv

In [2]:
data_folder = '../Data'
np.int64
time_aggregation = -1 # time in minutes. Use -1 to disable the time aggregation

In [3]:
dtype_dict = {
    'STARTTIMEMS': np.int64,
    'ENDTIMEMS': np.int64
}

df = pd.read_csv(f'{data_folder}/2023.csv').dropna().astype(dtype_dict)
df.head()

,STARTSTATIONNAME,STARTSTATIONARRONDISSEMENT,STARTSTATIONLATITUDE,STARTSTATIONLONGITUDE,ENDSTATIONNAME,ENDSTATIONARRONDISSEMENT,ENDSTATIONLATITUDE,ENDSTATIONLONGITUDE,STARTTIMEMS,ENDTIMEMS
0,Métro Mont-Royal (Utilités publiques / Rivard),Le Plateau-Mont-Royal,45.524236,-73.581552,Chabot / Mont-Royal,Le Plateau-Mont-Royal,45.534134,-73.573524,1698266696468,1698267092113
1,Rielle / Wellington,Verdun,45.460156,-73.567001,St-Jacques / McGill,Ville-Marie,45.501441,-73.560144,1698270535502,1698271967951
2,Ste-Catherine / Drummond,Ville-Marie,45.498588,-73.574278,Peel / Ottawa,Le Sud-Ouest,45.494200,-73.559985,1698285705974,1698285992418
3,Boyer / du Mont-Royal,Le Plateau-Mont-Royal,45.527432,-73.579917,de l'Hôtel-de-Ville / Rachel,Le Plateau-Mont-Royal,45.519897,-73.580106,1698269489932,1698269814383
4,Clark / Ontario,Ville-Marie,45.510625,-73.566903,Wolfe / Robin,Ville-Marie,45.519581,-73.560116,1698270006571,1698270440298


Extract and format nodes

In [4]:
df_start = df[['STARTSTATIONNAME', 'STARTSTATIONLATITUDE', 'STARTSTATIONLONGITUDE']].rename(columns={'STARTSTATIONNAME': 'name',
                                                                                                     'STARTSTATIONLATITUDE': 'latitude:float',
                                                                                                     'STARTSTATIONLONGITUDE': 'longitude:float'})
df_end = df[['ENDSTATIONNAME', 'ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE']].rename(columns={'ENDSTATIONNAME': 'name',
                                                                                             'ENDSTATIONLATITUDE': 'latitude:float',
                                                                                             'ENDSTATIONLONGITUDE': 'longitude:float'})
df_start.head()

,name,latitude:float,longitude:float
0,Métro Mont-Royal (Utilités publiques / Rivard),45.524236,-73.581552
1,Rielle / Wellington,45.460156,-73.567001
2,Ste-Catherine / Drummond,45.498588,-73.574278
3,Boyer / du Mont-Royal,45.527432,-73.579917
4,Clark / Ontario,45.510625,-73.566903


In [5]:
nodes = pd.concat([df_start, df_end], ignore_index=True).drop_duplicates()#.dropna()

print(nodes.shape)

(939, 3)


In [6]:
nodes["stationId:ID"] = nodes.index
nodes[":LABEL"] = "Station"
nodes = nodes[["stationId:ID", "name", "latitude:float", "longitude:float", ":LABEL"]]

In [7]:
nodes.head()

,stationId:ID,name,latitude:float,longitude:float,:LABEL
0,0,Métro Mont-Royal (Utilités publiques / Rivard),45.524236,-73.581552,Station
1,1,Rielle / Wellington,45.460156,-73.567001,Station
2,2,Ste-Catherine / Drummond,45.498588,-73.574278,Station
3,3,Boyer / du Mont-Royal,45.527432,-73.579917,Station
4,4,Clark / Ontario,45.510625,-73.566903,Station


In [8]:
nodes['stationId:ID'] = nodes['stationId:ID'].astype(int)
nodes.to_csv(f'{data_folder}/2023_nodes.csv', index=False, header=False)

In [9]:
nodes_header = ['stationId:ID','name','latitude:float','longitude:float',':LABEL']
with open(f'{data_folder}/2023_nodes_header.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(nodes_header)

Extract and format edges

In [10]:
# nodes = nodes.drop(columns=[':LABEL'])
# nodes.head()


edges = pd.merge(df, nodes,
                 left_on=['STARTSTATIONNAME', 'STARTSTATIONLATITUDE', 'STARTSTATIONLONGITUDE'],
                 right_on=['name', 'latitude:float', 'longitude:float'], how='left')

edges = edges[['ENDSTATIONNAME', 'ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE', 'STARTTIMEMS', 'STARTSTATIONLATITUDE', 'STARTSTATIONLONGITUDE', 'ENDTIMEMS', 'stationId:ID']].rename(columns={'stationId:ID': ":START_ID"})

edges = pd.merge(edges, nodes,
                 left_on=['ENDSTATIONNAME', 'ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE'], 
                 right_on=['name', 'latitude:float', 'longitude:float'], how='left')
# edges.head()
edges = edges[[':START_ID', 
               'STARTTIMEMS', 
               'STARTSTATIONLATITUDE', 
               'STARTSTATIONLONGITUDE',
               'ENDTIMEMS', 
               'ENDSTATIONLATITUDE', 
               'ENDSTATIONLONGITUDE', 
               'stationId:ID']].rename(columns={'stationId:ID': ":END_ID", 
                                                'STARTTIMEMS':'starttimems:int', 
                                                'STARTSTATIONLATITUDE':'startlatitude:float',
                                                'STARTSTATIONLONGITUDE':'startlongitude:float', 
                                                'ENDSTATIONLATITUDE':'endlatitude:float',
                                                'ENDSTATIONLONGITUDE':'endlongitude:float',
                                                'ENDTIMEMS':'endtimems:int'})


to_date = lambda ms: datetime.datetime.utcfromtimestamp(ms/1000).strftime('%Y-%m-%dT%H:%M:%S.%f')
edges = edges.dropna()
edges['start_date:DATETIME'] = edges['starttimems:int'].apply(to_date)
edges['end_date:DATETIME'] = edges['endtimems:int'].apply(to_date)
edges[":TYPE"] = "CYCLES_TO"
edges.head()

,:START_ID,starttimems:int,startlatitude:float,startlongitude:float,endtimems:int,endlatitude:float,endlongitude:float,:END_ID,start_date:DATETIME,end_date:DATETIME,:TYPE
0,0,1698266696468,45.524236,-73.581552,1698267092113,45.534134,-73.573524,505,2023-10-25T20:44:56.468000,2023-10-25T20:51:32.113000,CYCLES_TO
1,1,1698270535502,45.460156,-73.567001,1698271967951,45.501441,-73.560144,297,2023-10-25T21:48:55.502000,2023-10-25T22:12:47.951000,CYCLES_TO
2,2,1698285705974,45.498588,-73.574278,1698285992418,45.494200,-73.559985,467,2023-10-26T02:01:45.974000,2023-10-26T02:06:32.418000,CYCLES_TO
3,3,1698269489932,45.527432,-73.579917,1698269814383,45.519897,-73.580106,420,2023-10-25T21:31:29.932000,2023-10-25T21:36:54.383000,CYCLES_TO
4,4,1698270006571,45.510625,-73.566903,1698270440298,45.519581,-73.560116,1630,2023-10-25T21:40:06.571000,2023-10-25T21:47:20.298000,CYCLES_TO


Aggregate per x time

In [11]:
# I don't understand?
if time_aggregation > 0:
    df.groupby('category').size().reset_index(name='count')
    edges['starttimems:int'] = edges['starttimems:int'] // (time_aggregation * 60 * 1000)
    edges['endtimems:int'] = edges['endtimems:int'] // (time_aggregation * 60 * 1000)

In [12]:
print(edges.dtypes)

:START_ID                 int32
starttimems:int           int64
startlatitude:float     float64
startlongitude:float    float64
endtimems:int             int64
endlatitude:float       float64
endlongitude:float      float64
:END_ID                   int32
start_date:DATETIME      object
end_date:DATETIME        object
:TYPE                    object
dtype: object


In [13]:
# dtype_mapping = {':START_ID': int, 'starttimems':int, 'endtimems':int, ':END_ID':int, ':TYPE':str}
# edges = pd.DataFrame(edges, dtype=dtype_mapping)
edges[':START_ID'] = edges[':START_ID'].astype('int64')
edges[':END_ID'] = edges[':END_ID'].astype('int64')
edges['starttimems:int'] = edges['starttimems:int'].astype('int64')
edges['endtimems:int'] = edges['endtimems:int'].astype('int64')
edges.to_csv(f'{data_folder}/2023_edges.csv', index=False, header=False)

In [14]:
edges_header = [':START_ID','starttimems:int', 'startlatitude:float', 'startlongitude:float','endtimems:int', 'endlatitude:float', 'endlongitude:float',':END_ID','start_date:DATETIME','end_date:DATETIME',':TYPE']
with open(f'{data_folder}/2023_edges_header.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(edges_header)

In [15]:
edges.head()

,:START_ID,starttimems:int,startlatitude:float,startlongitude:float,endtimems:int,endlatitude:float,endlongitude:float,:END_ID,start_date:DATETIME,end_date:DATETIME,:TYPE
0,0,1698266696468,45.524236,-73.581552,1698267092113,45.534134,-73.573524,505,2023-10-25T20:44:56.468000,2023-10-25T20:51:32.113000,CYCLES_TO
1,1,1698270535502,45.460156,-73.567001,1698271967951,45.501441,-73.560144,297,2023-10-25T21:48:55.502000,2023-10-25T22:12:47.951000,CYCLES_TO
2,2,1698285705974,45.498588,-73.574278,1698285992418,45.494200,-73.559985,467,2023-10-26T02:01:45.974000,2023-10-26T02:06:32.418000,CYCLES_TO
3,3,1698269489932,45.527432,-73.579917,1698269814383,45.519897,-73.580106,420,2023-10-25T21:31:29.932000,2023-10-25T21:36:54.383000,CYCLES_TO
4,4,1698270006571,45.510625,-73.566903,1698270440298,45.519581,-73.560116,1630,2023-10-25T21:40:06.571000,2023-10-25T21:47:20.298000,CYCLES_TO


In [16]:
print(df.shape, "vs.", edges.shape)

(11719685, 10) vs. (11719685, 11)
